In [1]:
#   ▄████████    ▄████████ ████████▄  ████████▄   ▄█      ███             ▄████████    ▄████████  ▄████████    ▄█    █▄     ▄█   ▄█    █▄     ▄████████    ▄████████
#   ███    ███   ███    ███ ███   ▀███ ███   ▀███ ███  ▀█████████▄        ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ███    █▀  ███    ███ ███    ███ ███▌    ▀███▀▀██        ███    ███   ███    ███ ███    █▀    ███    ███   ███▌ ███    ███   ███    █▀    ███    ███
#  ▄███▄▄▄▄██▀  ▄███▄▄▄     ███    ███ ███    ███ ███▌     ███   ▀        ███    ███  ▄███▄▄▄▄██▀ ███         ▄███▄▄▄▄███▄▄ ███▌ ███    ███  ▄███▄▄▄      ▄███▄▄▄▄██▀
# ▀▀███▀▀▀▀▀   ▀▀███▀▀▀     ███    ███ ███    ███ ███▌     ███          ▀███████████ ▀▀███▀▀▀▀▀   ███        ▀▀███▀▀▀▀███▀  ███▌ ███    ███ ▀▀███▀▀▀     ▀▀███▀▀▀▀▀
# ▀███████████   ███    █▄  ███    ███ ███    ███ ███      ███            ███    ███ ▀███████████ ███    █▄    ███    ███   ███  ███    ███   ███    █▄  ▀███████████
#   ███    ███   ███    ███ ███   ▄███ ███   ▄███ ███      ███            ███    ███   ███    ███ ███    ███   ███    ███   ███  ███    ███   ███    ███   ███    ███
#   ███    ███   ██████████ ████████▀  ████████▀  █▀      ▄████▀          ███    █▀    ███    ███ ████████▀    ███    █▀    █▀    ▀██████▀    ██████████   ███    ███
#   ███    ███                                                                         ███    ███                                                          ███    ███

# Reddit Archiver - An easy way to archive and view your Reddit Account. 

Downloads your profile info and posts/comments from your input reddit account. Creates a presentable webpage of your arhcived profile that can be saved locally. 

* Allows user to input desired Reddit Username and output directory for archive file via command line
* Downloads user data using BeautifulSoup.
* Saves data to JSON file
* Loads JSON File into HTML webpage and saves webpage to desired directory


By [dandex200](https://github.com/dandex200/) on Github


#### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from tkinter import filedialog, Tk
import datetime
import time
import shutil

## User Input

In [2]:
#Username Input
user_input = input("Enter desired Reddit Username to Archive:")

#Output Directory Input using Tkinter
confirm_input = input("Choose Archive File Output Directory? (Y/N)")

#Select Output Directory if chosen
if confirm_input.lower() == 'y':
    root = Tk() 
    root.withdraw()
    output_dir = filedialog.askdirectory()
    root.destroy()
    print("Will save archive file to: %s" % output_dir)
else:
    print("No output directory selected - will save to current directory.")
    

Will save archive file to: /home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files


## Scraping

In [3]:
user_url = 'https://old.reddit.com/user/'+user_input

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

form_data = {"over18": "yes"}

response = requests.get(user_url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

title_check = soup.title.text.split(' ')[0]

if title_check == "overview":
    print("working")
else:
    print("Over 18, bypassing...")
    age_url =soup.select_one('link[rel="canonical"]')['href']
    print(age_url)
    new_response = requests.post(age_url, data=form_data, headers=headers)
    soup = BeautifulSoup(new_response.content, 'html.parser')
    title_check = soup.title.text.split(' ')[0]
    if title_check == "overview":
        print("bypass working")
    else:
        print("bypass failed")

working


### Profile Info

In [4]:
username = soup.title.text.split(' ')[2]

comment_karma_box = soup.find('span', {'class': 'comment-karma'})
comment_karma = comment_karma_box.text if comment_karma_box is not None else ''

post_karma = soup.find('span', {'class': 'karma'})
post_karma = post_karma.text if post_karma is not None else ''


user_creation_date = soup.find('span', {'class': 'age'})
user_creation_date = user_creation_date.find('time')['datetime']

num_posts = 0
num_comments = 0

print(username)
print(comment_karma)
print(post_karma)
print(user_creation_date)
print(num_posts)
print(num_comments)

SwipeGod4713
796
11,672
2021-04-12T20:18:33+00:00
0
0


### Comments

In [5]:
comments = []
comment_count = 0
spans = []
score = ""
score_hidden = ""
date_format = "%Y-%m-%dT%H:%M:%S%z"

for comment in soup.find_all('div', class_='comment'):
    utc_date = comment.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    spans = comment.find('span', class_='score unvoted')
    if spans is not None:
        score = spans.text
    else:
        score = comment.find('span', class_='score-hidden').text
    subreddit = comment.find('a', class_="subreddit").text
    permalink = comment.find('a', class_="bylink")['href']
    text = comment.find('div', class_='md-container').text
    comment_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'text': text
    }
    comments.append(comment_data)
    comment_count += 1


print(comments)
print(comment_count)

[{'utc_date': '2023-07-18T21:01:25+00:00', 'score': '3 points', 'subreddit': 'Atlantology', 'permalink': 'https://old.reddit.com/r/Atlantology/comments/1538ulk/niggas_be_making_shit_so_easy_for_the_feds/jsi5p7g/', 'text': 'Yea I heard his shit Lab Diamonds Nigga got great marketing🤣\n\n'}, {'utc_date': '2023-07-18T20:59:37+00:00', 'score': '12 points', 'subreddit': 'Atlantology', 'permalink': 'https://old.reddit.com/r/Atlantology/comments/1538ulk/niggas_be_making_shit_so_easy_for_the_feds/jsi5euk/', 'text': 'We All know He a Ripper he be running off tryna sell fake methods\n\n'}, {'utc_date': '2023-07-18T20:28:32+00:00', 'score': '1 point', 'subreddit': 'nope', 'permalink': 'https://old.reddit.com/r/nope/comments/152p8gs/indian_trains_are_crazy/jsi0kky/', 'text': 'No deodorant on there forsure\n\n'}, {'utc_date': '2023-07-18T15:31:41+00:00', 'score': '2 points', 'subreddit': 'Atlantology', 'permalink': 'https://old.reddit.com/r/Atlantology/comments/152kxul/this_the_worst_design_i_ever_

### Posts

In [6]:
#Counting Posts

post_count = 0
for post in soup.find_all('div', class_='link'):
    print(post)
    post_count += 1
print(post_count)

<div class="thing id-t3_153i6z5 odd link" data-author="SwipeGod4713" data-author-fullname="t2_bhypw90f" data-comments-count="18" data-context="listing" data-domain="v.redd.it" data-fullname="t3_153i6z5" data-gildings="0" data-is-gallery="false" data-kind="video" data-nsfw="false" data-num-crossposts="0" data-oc="false" data-permalink="/r/Atlantology/comments/153i6z5/first_von_now_this_smh_i_see_why_these_dudes_cant/" data-promoted="false" data-rank="" data-score="5" data-spoiler="false" data-subreddit="Atlantology" data-subreddit-fullname="t5_2wlawa" data-subreddit-prefixed="r/Atlantology" data-subreddit-type="public" data-timestamp="1689733953000" data-type="link" data-url="https://v.redd.it/mtt1uwgj2ucb1" data-whitelist-status="" id="thing_t3_153i6z5" onclick="click_thing(this)"><p class="parent"></p><span class="rank"></span><div class="midcol unvoted"><div aria-label="upvote" class="arrow up login-required access-required" data-event-action="upvote" role="button" tabindex="0"></div

In [7]:
#Identify Posts

posts = []
post_count = 0

utc_date = 0
post_type = ""
subreddit = ""
post_title = ""
file_path = ""
permalink = ""
score = 0
text = ""

#self post
for self_post in soup.find_all('div', class_='self'):
    post_count += 1
    utc_date = self_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "self"
    subreddit = self_post.find('a', class_="subreddit").text
    post_title = self_post.find('a', class_="title").text
    score = self_post.find('div', {'class': 'score unvoted'}).text
    permalink = self_post.find('a', class_="bylink")['href']

    #Self text post (hardest part so far)
    if self_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        #print("self expando")
        self_text_url = permalink
        print(self_text_url)

        headers2 = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response2 = requests.get(self_text_url, headers=headers2)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        post_content = soup2.find("div", class_="entry")
        md_element = post_content.find("div", class_="md-container")
        post_text = md_element.text
        #expando = soup2.find("div", class_="usertext-body")
        #md_element = expando.find("div", class_="md-container")

        #md_element = expando.find("div", class_="md-container") if expando else None
        #md_text = md_element.text if md_element else None

        #post_text = md_text.text

        text = post_text
        print(post_text)
        
    else:
        text = "gay"
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
    
#crosspost
for cross_post in soup.find_all('div', {'class':'link', 'data-crosspost-root-title': True}):
    post_count += 1
    utc_date = cross_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "crosspost"
    subreddit = cross_post.find('a', class_="subreddit").text
    post_title = cross_post.find('a', class_="title").text
    score = cross_post.find('div', {'class': 'score unvoted'}).text
    permalink = cross_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#link
domains = ['old.reddit.com', 'i.redd.it', 'v.redd.it', 'i.imgur.com']
for link_post in soup.find_all(lambda tag: tag.has_attr('data-domain') and not any(domain in tag['data-domain'] for domain in domains )and 'self' not in tag.get('class', [])):
    post_count += 1
    utc_date = link_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "link"
    subreddit = link_post.find('a', class_="subreddit").text
    post_title = link_post.find('a', class_="title").text
    score = link_post.find('div', {'class': 'score unvoted'}).text
    permalink = link_post.find('a', class_="bylink")['href']
    
    #If expando, request permalink page, extract text, else text = NaN
    
    if link_post.find('div', {'class': 'expando-button'}): #If expando button present
        #Download permalink
        form_data = {"over18": "yes"}
        response = requests.get(permalink, data=form_data, headers={'User-Agent': 'Mozilla/5.0'})
        post_soup = BeautifulSoup(response.content, 'html.parser')
        #Extract text
        text = post_soup.find('div', {'class': 'entry'}).find('div', {'class': 'md-container'}).get_text()
    else:
        text = None
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
#video
for video_post in soup.find_all('div', {'class':'link', "data-domain": "v.redd.it"}):
    post_count += 1
    utc_date = video_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "video"
    subreddit = video_post.find('a', class_="subreddit").text
    post_title = video_post.find('a', class_="title").text
    score = video_post.find('div', {'class': 'score unvoted'}).text
    permalink = video_post.find('a', class_="bylink")['href']
    text = None
    vid_link = video_post.find('a', class_="title")['href']
    print(vid_link)
    vid_response = requests.get(vid_link, stream=True)
    output_file = "vid1.mp4"
    if vid_response.status_code == 200:
        with open(Temp_Files/output_file, 'wb') as file:
            vid_response.raw.decode_content = True
            shutil.copyfileobj(vid_response.raw, file)
        print("Video saved successfully.")
    else:
        print("Error downloading the video.")
    vid_path = ""
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text, 
        'filepath': vid_path
    }
    posts.append(post_data)
#picture 
for pic_post in soup.find_all('div', {'class':'link', "data-domain": "old.reddit.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    pic_link = pic_post.find('a', class_="title")['href']
    print(pic_link)
    pic_path = ""
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text,
        'filepath': pic_path
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.redd.it"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    pic_link = pic_post.find('a', class_="title")['href']
    print(pic_link)
    pic_path = ""
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text, 
        'filepath': pic_path
    }
    posts.append(post_data)

for pic_post in soup.find_all('div', {'class':'link', "data-domain": "i.imgur.com"}):
    post_count += 1
    utc_date = pic_post.find('time', class_="live-timestamp")['datetime']
    utc_date = datetime.datetime.strptime(utc_date, date_format)
    utc_date = utc_date.isoformat()
    post_type = "pic"
    subreddit = pic_post.find('a', class_="subreddit").text
    post_title = pic_post.find('a', class_="title").text
    score = pic_post.find('div', {'class': 'score unvoted'}).text
    permalink = pic_post.find('a', class_="bylink")['href']
    post_data = {
        'utc_date': utc_date,
        'score': score,
        'subreddit': subreddit,
        'permalink': permalink,
        'title': post_title,
        'type': post_type,
        'text': text
    }
    posts.append(post_data)
print(post_count)

https://v.redd.it/mtt1uwgj2ucb1
Video saved successfully.
https://v.redd.it/kiwvtik9nxbb1
Video saved successfully.
https://i.redd.it/a4smau0q5scb1.jpg
https://i.redd.it/1xpyh1gb7gcb1.jpg
https://i.redd.it/ksoqbea9tqbb1.jpg
5


In [8]:
#Test if 25 Posts identified (Number of each page)
if (post_count + comment_count) != 25:
    print("Error")
else:
    print("working")

working


In [9]:
num_posts = len(posts)
num_comments = len(comments)

In [10]:
user_data = {
    'user_info': {
        'username': username,
        'user_creation_date': user_creation_date,
        'num_posts': num_posts,
        'num_comments': num_comments,
        'karma': {
            'post_karma': post_karma,
            'comment_karma': comment_karma
        }
    },
    'posts': posts,
    'comments': comments
}

In [11]:
with open('Temp_Files/user_data.json', 'w') as f:
    json.dump(user_data, f)

In [12]:
with open('Temp_Files/user_data.json') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))


{
    "user_info": {
        "username": "SwipeGod4713",
        "user_creation_date": "2021-04-12T20:18:33+00:00",
        "num_posts": 5,
        "num_comments": 20,
        "karma": {
            "post_karma": "11,672",
            "comment_karma": "796"
        }
    },
    "posts": [
        {
            "utc_date": "2023-07-19T02:32:33+00:00",
            "score": "5",
            "subreddit": "r/Atlantology",
            "permalink": "https://old.reddit.com/r/Atlantology/comments/153i6z5/first_von_now_this_smh_i_see_why_these_dudes_cant/",
            "title": "First Von Now this SMH I see why these Dudes Can\u2019t stop going to Jail",
            "type": "video",
            "text": null,
            "filepath": ""
        },
        {
            "utc_date": "2023-07-14T13:29:59+00:00",
            "score": "14",
            "subreddit": "r/Atlantology",
            "permalink": "https://old.reddit.com/r/Atlantology/comments/14zgiot/how_barbershops_in_atl_be_nowadays_smh/",


**To-DO: PT1**
* Successfully append comments, user info, and posts of first page to JSON in correct format
* Save pictures/videos to directory
* Present JSON and saved files in beginner format HTML webpage
* Apply saving to all pages of reddit account
* Add tests to ensure data is all saved (meets 25 posts per page, etc)
* Create Readme on Github
* Publish first time

**TO-DO PT2**
* Convert from notebook to script
* Create GUI where you can choose if you want to save pics/videos, input user/output directory in GUI, etc
* Apply ability to save pics/videos as choice
* Create Pretty looking HTML archive page
* Promote on Reddit
* Release as second update 
* Add ability to keep link in comment

**TO-DO PT3**
* blank

**Minor Additions**
* Accounting for pinned posts
* Profile pictures (snoovatar??)
* Trophies
* Add over 18 bypass for old.reddit

In [47]:
output_dir = "/home/dandex/programming/The_Reddit_Archiver/Reddit_Archiver/Temp_Files/"

In [48]:
#Clear Temp_Files
for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)